# Aula 1 - Código de Bloco

# Parte 1

## 1. Um codificador de canal para o código de Hamming como descrito.

In [59]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [60]:
# Defining a function as a Hamming coder
def hamming_coder(input_vector: np.ndarray) -> np.ndarray:
    # Defining a parity matrix
    G = np.array([[1, 0, 0, 0, 1, 1, 1],
                  [0, 1, 0, 0, 1, 0, 1],
                  [0, 0, 1, 0, 1, 1, 0],
                  [0, 0, 0, 1, 0, 1, 1]], dtype=int)
    # Encoding the input vector
    output_vector = np.dot(input_vector, G) % 2
    return output_vector

In [61]:
input = np.array([0, 1, 1, 0])
output = hamming_coder(input)
output

array([0, 1, 1, 0, 0, 1, 1], dtype=int32)

## 2. Um canal BSC com parâmetro p

In [62]:
# Define a function for a Binary Symmetric Channel with parameter p
def bsc(input_vector: np.ndarray, p: float) -> np.ndarray:
    # Generate a random vector with the same length as the input vector
    random_vector = np.random.rand(input_vector.shape[0])
    # Apply the channel to the input vector
    output_vector = (input_vector + (random_vector < p)) % 2
    return output_vector

In [63]:
# Test the bsc
input = np.array([0, 1, 1, 0])
output = bsc(input, 0.08)
output

array([0, 1, 0, 0], dtype=int32)

## 3. Um decodificador de canal para o codificador do item 1

### 3.1 Criando a função para se calcular a síndrome

In [64]:
# Define a function for the syndrome calculator
def syndrome(input_vector: np.ndarray) -> np.ndarray:
    # Defining a parity-check matrix
    H = np.array([[1, 1, 1, 0, 1, 0, 0],
                  [1, 0, 1, 1, 0, 1, 0],
                  [1, 1, 0, 1, 0, 0, 1]], dtype=int)
    # Decoding the input vector
    output_vector = np.dot(input_vector, H.T) % 2
    return output_vector

In [65]:
# Test the syndrome calculator
input = np.array([1, 0, 1, 1])
coded_input = hamming_coder(input)
coded_input

array([1, 0, 1, 1, 0, 1, 0], dtype=int32)

In [66]:
s = syndrome(coded_input)
s

array([0, 0, 0], dtype=int32)

### 3.2 Criando a associação para o síndrome encontrado

In [67]:
s_dict = {
    '0': (np.array([0, 0, 0]), np.array([0, 0, 0, 0, 0, 0, 0])),
    '1': (np.array([1, 1, 1]), np.array([1, 0, 0, 0, 0, 0, 0])),
    '2': (np.array([1, 0, 1]), np.array([0, 1, 0, 0, 0, 0, 0])),
    '3': (np.array([1, 1, 0]), np.array([0, 0, 1, 0, 0, 0, 0])),
    '4': (np.array([0, 1, 1]), np.array([0, 0, 0, 1, 0, 0, 0])),
    '5': (np.array([1, 0, 0]), np.array([0, 0, 0, 0, 1, 0, 0])),
    '6': (np.array([0, 1, 0]), np.array([0, 0, 0, 0, 0, 1, 0])),
    '7': (np.array([0, 0, 1]), np.array([0, 0, 0, 0, 0, 0, 1]))
}

In [68]:
# Calculating the e_prime from syndrome
def e_prime_calc(s: np.ndarray) -> np.ndarray:
    for s_index in s_dict.items():
        if np.array_equal(s_index[1][0], s):
            return s_index[1][1]

In [69]:
test = np.array([0, 0, 1, 0, 0, 0, 0])
s = syndrome(test)
s

array([1, 1, 0], dtype=int32)

In [70]:
e_prime = e_prime_calc(s)
e_prime

array([0, 0, 1, 0, 0, 0, 0])

### 3.3 Finalmente, criando a função para decodificar corrigindo o erro de um bit

In [71]:
# Defining the Hamming decoder function with 1-bit correction
def hamming_decoder(input_vector: np.ndarray) -> np.ndarray:
    s = syndrome(input_vector)
    e_prime = e_prime_calc(s)
    decoded = (input_vector + e_prime) % 2
    return decoded[:4]

In [72]:
test = np.array([0, 1, 1, 0, 0, 1, 1])
output_test = hamming_decoder(test)
output_test

array([0, 1, 1, 0], dtype=int32)

## 4. Um código e codificador de canal criado pelos próprios alunos, com taxa semelhante (±%10) ao do código de Hamming, mas com palavras-código de tamanho maior. Nao é suficiente simplesmente repetir o código de Hamming, ou algo semelhante.


In [73]:
def encode_custom(info_bits: np.ndarray) -> np.ndarray:
    code_word = np.zeros(9, dtype=int)
    code_word[:5] = info_bits
    code_word[5] = info_bits[1] ^ info_bits[2] ^ info_bits[3]	
    code_word[6] = info_bits[0] ^ info_bits[2] ^ info_bits[3]
    code_word[7] = info_bits[0] ^ info_bits[1] ^ info_bits[3]
    code_word[8] = sum(code_word[:8]) % 2
    return code_word

In [74]:
input = np.array([0, 1, 1, 0, 1])
output = encode_custom(input)
output

array([0, 1, 1, 0, 1, 0, 1, 1, 1])

## 5. Um decodificador para o codificador do item 4

In [75]:
# Insert code after
def decode_custom(code_word):
    parity_checks = np.zeros(4, dtype=int)
    parity_checks[0] = code_word[1] ^ code_word[2] ^ code_word[3] ^ code_word[5]
    parity_checks[1] = code_word[0] ^ code_word[2] ^ code_word[3] ^ code_word[6]
    parity_checks[2] = code_word[0] ^ code_word[1] ^ code_word[3] ^ code_word[7]
    parity_checks[3] = sum(code_word[:8]) % 2 ^ code_word[8]
    #print(parity_checks)
    
    if sum(parity_checks > 0):    
        if parity_checks[3] == 1:
            if sum(parity_checks) == 1:
                error_pos = 4
            if sum(parity_checks) == 2:
                if parity_checks[0] == 1:
                    error_pos = 5
                elif parity_checks[1] == 1:
                    error_pos = 6
                elif parity_checks[2] == 1:
                    error_pos = 7
            elif sum(parity_checks) == 3:
                error_pos = np.argmin(parity_checks)
            elif sum(parity_checks) == 4:
                error_pos = 3 
            code_word[error_pos] ^= 1
        else:
            if sum(parity_checks) == 1:
                error_pos = 3
                error_pos2 = np.argmax(parity_checks)
            elif sum(parity_checks) == 2:
                error_pos = 3
                if parity_checks[0] == 0:
                    error_pos2 = 5
                elif parity_checks[1] == 0:
                    error_pos2 = 6
                elif parity_checks[2] == 0:
                    error_pos2 = 7
            elif sum(parity_checks) == 3:
                error_pos = 0
                error_pos2 = 5
            code_word[error_pos] ^= 1
            code_word[error_pos2] ^= 1
            #print(error_pos2)


    
    #print(error_pos)

    return code_word[:5]

In [76]:
test = np.array([0, 1, 0, 0, 1])
input_test = encode_custom(test)
input_test

array([0, 1, 0, 0, 1, 1, 0, 1, 0])

In [77]:
input_test[3] ^= 1
#input_test[7] ^= 1
#input_test[3] ^= 1
input_test

array([0, 1, 0, 1, 1, 1, 0, 1, 0])

In [78]:
output_test = decode_custom(input_test)
output_test

array([0, 1, 0, 0, 1])

In [79]:
# Generate 1000 random vectors of 5 bits
num_samples = int(1e4)
input_data = np.random.randint(0, 2, (num_samples, 5))

# Encode the input data
encoded_data = np.zeros((num_samples, 9), dtype=int)
for i in range(num_samples):
    encoded_data[i] = encode_custom(input_data[i])

# Decode the encoded data
decoded_data = np.zeros((num_samples, 5), dtype=int)
for i in range(num_samples):
    decoded_data[i] = decode_custom(encoded_data[i])

# Compere the input data with the decoded data and count the errors
num_errors = 0
for i in range(num_samples):
    num_errors += np.sum(input_data[i] != decoded_data[i])

num_errors

0

# Parte 2

## 1. Calculando $\hat{p}$ a partir apenas da passagem por BSC

In [80]:
# Sample_size is defined to be 1 million -> check Chebyshev inequality after
def bsc_pb_est(p: float) -> float:
    sample_size = 1_000_000
    error_count = 0
    k = 1
    l = int(sample_size/k)

    for _ in range(l):
        input_vector = np.random.randint(0, 2, k)
        output_vector = bsc(input_vector, p)
        if not np.array_equal(input_vector, output_vector):
            error_count += 1

    return (1/l) * error_count        

In [81]:
bsc_pb_est(0.05)

0.050068999999999995

## 2. Calculando $\hat{p}$ a partir da codificação e passagem por BSC

In [82]:
def hamming_pb_est(p: float) -> float:
    sample_size = 1_000_000
    error_count = 0
    k = 4
    l = int(sample_size/k)

    for _ in range(l):
        input_vector = np.random.randint(0, 2, k)
        input_decoded = hamming_coder(input_vector)
        transmitted_vector = bsc(input_decoded, p)
        decoded_vector = hamming_decoder(transmitted_vector)
        if not np.array_equal(input_vector, decoded_vector):
            error_count += 1

    return (1/l) * error_count 

In [83]:
hamming_pb_est(0.001)

1.2e-05

## 3. Calculando $\hat{p}$ a partir da codificação custom e passagem por BSC

In [84]:
def custom_pb_est(p: float) -> float:
    sample_size = 1_000_000
    error_count = 0
    k = 5
    l = int(sample_size/k)

    for _ in range(l):
        input_vector = np.random.randint(0, 2, k)
        input_decoded = encode_custom(input_vector)
        transmitted_vector = bsc(input_decoded, p)
        decoded_vector = decode_custom(transmitted_vector)
        if not np.array_equal(input_vector, decoded_vector):
            error_count += 1

    return (1/l) * error_count 

In [85]:
custom_pb_est(0.001)

0.0010400000000000001

## 4. Fazendo os _plots_ dos resultados

In [86]:
p = np.linspace(0.5, 1e-3, 12)
error_results_bsc = [bsc_pb_est(p_i) for p_i in p]
error_results_hamming = [hamming_pb_est(p_i) for p_i in p]

In [87]:
error_results_custom = [custom_pb_est(p_i) for p_i in p]

In [89]:
# Make a log plot of p vs. the estimated error rate
matplotlib.font_manager._load_fontmanager(try_read_cache=False)
plt.rcParams['font.family'] = "CMU Serif"
plt.rcParams['text.usetex'] = True
plt.rcParams['axes.linewidth'] = 1.0
fig, ax = plt.subplots(figsize=(7, 5))
ax.plot(p, error_results_hamming, label="Hamming", color='red')
ax.plot(p, error_results_bsc, label="BSC", color='blue')
ax.plot(p, error_results_custom, label="Custom", color='green')
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend(loc = 0)
ax.set_xlabel('p')
ax.set_ylabel('Pb')
ax.set_title("Error Rate Estimation")
plt.gca().invert_xaxis()
plt.show()

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 700x500 with 1 Axes>